#### IMPORTACION LIBRERIAS

In [13]:
from pycuda.compiler import SourceModule
import pycuda.driver as cuda
from pycuda import gpuarray
import numpy as np
import base64
import math
import time

#### FUNCIONES GENERALES

In [14]:
def leer_archivo(ruta):
    with open(ruta, 'rb') as archivo:
        return archivo.read()

def escribir_archivo(ruta, contenido):
    if isinstance(contenido, bytes):
        with open(ruta, 'wb') as archivo:
            archivo.write(contenido)
    elif isinstance(contenido, list):
        contenido_str = ' '.join(map(str, contenido))
        with open(ruta, 'w', encoding='utf-8') as archivo:
            archivo.write(contenido_str)
    else:
        with open(ruta, 'w', encoding='utf-8') as archivo:
            archivo.write(contenido)
def conversion_ascii(texto):
    return np.array([ord(char) for char in texto], dtype=np.int32)

def conversion_texto(lista_ascii):
    return ''.join(chr(num) for num in lista_ascii)

#### ENCRIPTADO SECUECNCIAL EN PYTHON AES

In [15]:
def encriptar_aes_secuencial(texto):
    if isinstance(texto, str):
        texto_np = np.frombuffer(texto.encode('utf-8'), dtype=np.uint8)
    else:
        texto_np = np.frombuffer(texto, dtype=np.uint8)
    texto_encriptado = bytearray()

    base_key = 0xBB
    for idx, byte in enumerate(texto_np):
        xor_key = base_key ^ (idx % 256)
        byte = byte ^ xor_key
        byte = ((byte >> 2) | (byte << (8 - 2))) & 0xFF 
        mask = 0x3F
        mask ^= (idx & 0xFF)
        byte = (byte ^ mask) & 0xFF
        texto_encriptado.append(byte)

    return bytes(texto_encriptado)

def desencriptar_aes_secuencial(texto_ecncriptado):
    if isinstance(texto_ecncriptado, str):
        texto_ecncriptado = texto_ecncriptado.encode('utf-8')
    texto_np = np.frombuffer(texto_ecncriptado, dtype=np.uint8)
    texto_descifrado = bytearray()

    base_key = 0xBB
    for idx, byte in enumerate(texto_np):
        mask = 0x3F
        mask ^= (idx & 0xFF)
        byte = (byte ^ mask) & 0xFF
        byte = ((byte << 2) | (byte >> (8 - 2))) & 0xFF 
        xor_key = base_key ^ (idx % 256)
        byte = (byte ^ xor_key) & 0xFF  
        texto_descifrado.append(byte)

    return bytes(texto_descifrado).decode('utf-8', errors='ignore')


ruta_archivo = 'C:/Users/marco/Downloads/Dracula.txt'
ruta_archivo_encriptado_cpu_aes = 'C:/Users/marco/Downloads/texto_encriptado_cpu_aes.txt'
ruta_archivo_desencriptado_cpu_aes = 'C:/Users/marco/Downloads/texto_desencriptado_cpu_aes.txt'

texto_original = leer_archivo(ruta_archivo)
inicio = time.time()
texto_encriptado = encriptar_aes_secuencial(texto_original)
fin = time.time()
print("Tiempo de encriptación:", fin - inicio, "segundos")
texto_encriptado_b64 = base64.b64encode(texto_encriptado).decode('utf-8')
escribir_archivo(ruta_archivo_encriptado_cpu_aes, texto_encriptado_b64)

texto_encriptado_b64 = leer_archivo(ruta_archivo_encriptado_cpu_aes)
texto_encriptado = base64.b64decode(texto_encriptado_b64)
inicio = time.time()
texto_desencriptado = desencriptar_aes_secuencial(texto_encriptado)
fin = time.time()
print("Tiempo de desencriptación:", fin - inicio, "segundos")
escribir_archivo(ruta_archivo_desencriptado_cpu_aes, texto_desencriptado)

Tiempo de encriptación: 1.5585341453552246 segundos
Tiempo de desencriptación: 1.5937271118164062 segundos


#### ENCRIPTADO GPU EN PYCUDA AES

In [16]:
def encriptar_aes_pycuda(texto):
    cuda.init()
    device = cuda.Device(0)
    context = device.make_context()
    try:
        texto_np = np.frombuffer(texto.encode('utf-8'), dtype=np.uint8)
        texto_gpu = gpuarray.to_gpu(texto_np)

        encriptacion_aes = """
            __global__ void encriptacion_aes(unsigned char *data, int data_size) {
                int idx = threadIdx.x + blockIdx.x * blockDim.x;
                if (idx < data_size) {
                    unsigned char base_key = 0xBB;
                    unsigned char xor_key = base_key ^ (idx % 256);
                    data[idx] = data[idx] ^ xor_key;
                    data[idx] = (data[idx] >> 2) | (data[idx] << (8 - 2));
                    unsigned char mask = 0x3F; 
                    mask ^= (idx & 0xFF); 
                    data[idx] ^= mask;
                }
            }
        """
        mod = SourceModule(encriptacion_aes)
        aes_kernel = mod.get_function("encriptacion_aes")

        hilos_bloque = 256
        bloques = math.ceil(len(texto_np)/hilos_bloque)

        aes_kernel(texto_gpu, np.int32(texto_np.size), block=(hilos_bloque, 1, 1), grid=(bloques, 1))

        context.synchronize()

        texto_encriptado = texto_gpu.get().tobytes()
        return texto_encriptado
    finally:
        context.pop()
        context.detach()

def desencriptar_aes_pycuda(texto_ecncriptado):
    cuda.init()
    device = cuda.Device(0)
    context = device.make_context()
    try:
        texto_gpu = gpuarray.to_gpu(np.frombuffer(texto_ecncriptado, dtype=np.uint8))

        desencriptacion_aes = """
            __global__ void desencriptacion_aes(unsigned char *data, int data_size) {
                int idx = threadIdx.x + blockIdx.x * blockDim.x;
                if (idx < data_size) {
                    unsigned char mask = 0x3F; 
                    mask ^= (idx & 0xFF);
                    data[idx] ^= mask; 
                    data[idx] = (data[idx] << 2) | (data[idx] >> (8 - 2));

                    unsigned char base_key = 0xBB;
                    unsigned char xor_key = base_key ^ (idx % 256);
                    data[idx] = data[idx] ^ xor_key;
                }
            }
        """

        mod = SourceModule(desencriptacion_aes)
        aes_kernel = mod.get_function("desencriptacion_aes")
        hilos_bloque = 256
        bloques = math.ceil(texto_gpu.size / hilos_bloque)

        aes_kernel(texto_gpu, np.int32(texto_gpu.size), block=(hilos_bloque, 1, 1), grid=(bloques, 1))

        resultado_texto = texto_gpu.get().tobytes().decode('utf-8', errors='ignore')
        return resultado_texto
    finally:
        context.pop()
        context.detach()

ruta_archivo_original = 'C:/Users/marco/Downloads/Dracula.txt'
ruta_archivo_encriptado_gpu_aes = 'C:/Users/marco/Downloads/texto_encriptado_gpu_aes.txt'
ruta_archivo_desencriptado_gpu_aes = 'C:/Users/marco/Downloads/texto_desencriptado_gpu_aes.txt'

texto_original_binario = leer_archivo(ruta_archivo_original)
texto_original = texto_original_binario.decode('utf-8')

inicio = time.time()
texto_encriptado = encriptar_aes_pycuda(texto_original)
fin = time.time()
print("Tiempo de encriptación:", fin - inicio, "segundos")
escribir_archivo(ruta_archivo_encriptado_gpu_aes, texto_encriptado)

texto_encriptado_binario = base64.b64decode(leer_archivo(ruta_archivo_encriptado_gpu_aes))

inicio = time.time()
texto_desencriptado = desencriptar_aes_pycuda(texto_encriptado_binario)
fin = time.time()
print("Tiempo de desencriptación:", fin - inicio, "segundos")
escribir_archivo(ruta_archivo_desencriptado_gpu_aes, texto_desencriptado)

Tiempo de encriptación: 0.0982062816619873 segundos
Tiempo de desencriptación: 0.08228206634521484 segundos


#### ENCRIPTADO SECUECNCIAL EN PYTHON RSA

In [17]:
def exp_mod(base, exp, mod):
    resultado = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:
            resultado = (resultado * base) % mod
        exp = exp // 2
        base = (base * base) % mod
    return resultado

def encriptar_rsa_secuencial(texto, e=17, n=3233):
    texto_ascii = conversion_ascii(texto)
    texto_encriptado = [exp_mod(char, e, n) for char in texto_ascii]
    return texto_encriptado

def desencriptar_rsa_secuencial(texto_ecncriptado, d=2753, n=3233):
    texto_desencriptado_ascii = [exp_mod(char, d, n) for char in texto_ecncriptado]
    texto_desencriptado = conversion_texto(texto_desencriptado_ascii)
    return texto_desencriptado

ruta_archivo_original = 'C:/Users/marco/Downloads/Dracula.txt'
ruta_archivo_encriptado_rsa = 'C:/Users/marco/Downloads/texto_encriptado_cpu_rsa.txt'
ruta_archivo_desencriptado_rsa = 'C:/Users/marco/Downloads/texto_desencriptado_cpu_rsa.txt'

texto_original_binario = leer_archivo(ruta_archivo_original)
texto_original = texto_original_binario.decode('utf-8')
inicio = time.time()
texto_encriptado_rsa = encriptar_rsa_secuencial(texto_original)
fin = time.time()
print("Tiempo de encriptación RSA:", fin - inicio, "segundos")
escribir_archivo(ruta_archivo_encriptado_rsa, texto_encriptado_rsa)

texto_encriptado_rsa_str = leer_archivo(ruta_archivo_encriptado_rsa).decode('utf-8')
texto_encriptado_rsa = list(map(int, texto_encriptado_rsa_str.split()))
inicio = time.time()
texto_desencriptado_rsa = desencriptar_rsa_secuencial(texto_encriptado_rsa)
fin = time.time()
print("Tiempo de desencriptación RSA:", fin - inicio, "segundos")
escribir_archivo(ruta_archivo_desencriptado_rsa, texto_desencriptado_rsa)

Tiempo de encriptación RSA: 0.93402099609375 segundos
Tiempo de desencriptación RSA: 1.326488733291626 segundos


#### ENCRIPTADO GPU EN PYCUDA RSA

In [18]:
def encriptar_rsa_pycuda(texto, e=17, n=3233):
    cuda.init()
    device = cuda.Device(0)
    context = device.make_context()
    try:
        texto_np = conversion_ascii(texto)
        texto_gpu = gpuarray.to_gpu(texto_np)
        encriptacion_rsa = """
            __device__ int exp_mod(int base, int exp, int mod) {
                int res = 1;
                base = base % mod;
                while (exp > 0) {
                    if (exp & 1) 
                        res = (res * base) % mod;
                    exp >>= 1; 
                    base = (base * base) % mod;
                }
                return res;
            }
            __global__ void encriptacion_rsa(int *valores, int clave, int modulo, int tam) {
                int i = blockIdx.x * blockDim.x + threadIdx.x;
                if (i < tam) {
                    valores[i] = exp_mod(valores[i], clave, modulo);
                }
            }
        """
        mod = SourceModule(encriptacion_rsa)
        rsa_kernel = mod.get_function("encriptacion_rsa")
        hilos_bloque = 256
        bloques = math.ceil(len(texto_np)/hilos_bloque)
        rsa_kernel(texto_gpu, np.int32(e), np.int32(n), np.int32(len(texto_np)), block=(hilos_bloque, 1, 1), grid=(bloques, 1))
        cuda.Context.synchronize()
        resultado = texto_gpu.get()
        return resultado.tolist()
    finally:
        context.pop()
        context.detach()

def desencriptar_rsa_pycuda(texto_ecncriptado, d=2753, n=3233):
    cuda.init()
    device = cuda.Device(0)
    context = device.make_context()
    try:
        texto_encriptado_np = np.array(texto_ecncriptado, dtype=np.int32)
        texto_encriptado_gpu = gpuarray.to_gpu(texto_encriptado_np)
        desencriptacion_rsa = """
            __device__ int exp_mod(int value, int power, int mod) {
                int result = 1;
                value = value % mod;
                while (power > 0) {
                    if (power & 1)
                        result = (long long)result * value % mod;
                    power >>= 1;
                    value = (long long)value * value % mod;
                }
                return result;
            }

            __global__ void descencriptacion_rsa(int *values, int exponent, int modulus, int length) {
                int index = blockIdx.x * blockDim.x + threadIdx.x;
                if (index < length) {
                    values[index] = exp_mod(values[index], exponent, modulus);
                }
            }
        """
        mod = SourceModule(desencriptacion_rsa)
        rsa_kernel = mod.get_function("descencriptacion_rsa")
        hilos_bloque = 256
        bloques = math.ceil(len(texto_ecncriptado) / hilos_bloque)
        rsa_kernel(texto_encriptado_gpu, np.int32(d), np.int32(n), np.int32(len(texto_ecncriptado)), block=(hilos_bloque, 1, 1), grid=(bloques, 1))
        cuda.Context.synchronize()
        resultado = texto_encriptado_gpu.get()
        texto_desencriptado = conversion_texto(resultado)
        return texto_desencriptado
    finally:
        context.pop()
        context.detach()

ruta_archivo_original = 'C:/Users/marco/Downloads/Dracula.txt'
ruta_archivo_encriptado_rsa = 'C:/Users/marco/Downloads/texto_encriptado_gpu_rsa.txt'
ruta_archivo_desencriptado_rsa = 'C:/Users/marco/Downloads/texto_desencriptado_gpu_rsa.txt'

texto_original_binario = leer_archivo(ruta_archivo_original)
texto_original = texto_original_binario.decode('utf-8')
inicio = time.time()
texto_encriptado_rsa = encriptar_rsa_pycuda(texto_original)
fin = time.time()
print("Tiempo de encriptación RSA con PyCUDA:", fin - inicio, "segundos")
escribir_archivo(ruta_archivo_encriptado_rsa, texto_encriptado_rsa)

texto_encriptado_rsa_str = leer_archivo(ruta_archivo_encriptado_rsa).decode('utf-8')
texto_encriptado_rsa = list(map(int, texto_encriptado_rsa_str.split()))
inicio = time.time()
texto_desencriptado_rsa = desencriptar_rsa_pycuda(texto_encriptado_rsa)
fin = time.time()
print("Tiempo de desencriptación RSA con PyCUDA:", fin - inicio, "segundos")
escribir_archivo(ruta_archivo_desencriptado_rsa, texto_desencriptado_rsa)

Tiempo de encriptación RSA con PyCUDA: 0.20597124099731445 segundos
Tiempo de desencriptación RSA con PyCUDA: 0.19343781471252441 segundos
